In [ ]:

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import pandas as pd

from matplotlib.ticker import MultipleLocator
pd.set_option('display.max_columns', None)
import os, requests, time
from google.colab import userdata
fred_api_key=userdata.get('FRED_API_KEY')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# I. Create dataset

## A. Load Functions

In [ ]:
def get_children_categories(category_id, api_key, parent_name="ROOT" ):
  store_list=[]
  url="https://api.stlouisfed.org/fred/category/children?"
  params = {
      "api_key":api_key,
      "category_id": category_id,
      "file_type": "json"
  }
  response = requests.get(url, params=params)
  if response.status_code == 200:
    data = response.json()
    children_categories = data.get('categories')
    for children_categorie in children_categories:
      # print(children_categorie)
      child_categori_id = children_categorie.get('id')
      child_name = children_categorie.get("name")
      store_list.append({
            "parent_id": children_categorie.get("parent_id"),
            "parent_name": parent_name,
            "category_id": child_categori_id,
            "name": children_categorie.get("name"),
            "notes": children_categorie.get("notes", "")
        })
  else:
      print(f"Error: {response.status_code}, category: {category_id}")
      return
  df=pd.DataFrame.from_dict(store_list)
  print(df)
  return df

def get_series_from_category(category_id, api_key, filter_term=None):
  store_list=[]
  url="https://api.stlouisfed.org/fred/category/series?"
  params = {
      "api_key":api_key,
      "category_id": category_id,
      "file_type": "json"
  }
  response = requests.get(url, params=params)
  if response.status_code == 200:
    data = response.json()

    series = data.get('seriess')
    for serie in series:
      store_list.append(serie)
      # serie_id = serie.get('id')
      # child_name = serie.get("name")
      # store_list.append({
      #       "parent_id": serie.get("parent_id"),
      #       "parent_name": parent_name,
      #       "category_id": serie_id,
      #       "name": serie.get("name"),
      #       "notes": serie.get("notes", "")
      #   })
  else:
      print(f"Error: {response.status_code}, category: {category_id}")
      return
  df=pd.DataFrame.from_dict(store_list).drop_duplicates('id')
  df.drop(['realtime_start', 'realtime_end','observation_start', 'observation_end','frequency_short', 'units_short',
           'seasonal_adjustment_short','last_updated', 'popularity', 'group_popularity'], axis=1, inplace=True)
  if filter_term:
    df=df[df.title.str.contains(filter_term, case=False, na=False)]
  return df


def search_series(search_name, api_key,frequency_criteria="Monthly", search_term=None):
  SEARCH_URL = f"https://api.stlouisfed.org/fred/series/search?"
  store_list = []
  params = {
      "api_key":api_key,
      "search_text":search_name,
      "file_type": "json"
  }
  response = requests.get(SEARCH_URL, params=params)
  if response.status_code == 200:
    data = response.json()
    # print(data.get("count"))
    series=data.get("seriess")
    for serie in series:
      serie_id = serie.get("id")
      title = serie.get("title")
      frequency=serie.get("frequency")
      note = serie.get("notes")
      unit = serie.get("units")
      if frequency==frequency_criteria:
        store_list.append({"serie_id":serie_id,
                           "title":title,
                           "unit": unit,
                           "notes": note,
                           "seasonal_adjustment":serie.get("seasonal_adjustment")
                           })
  else:
      print(f"Error: {response.status_code}")
      return
  df = pd.DataFrame(store_list)
  if search_term:
    df=df[df.title.str.contains(search_term, case=False, na=False)]
  return df

def get_time_series(serie_id, api_key,start_time="1776-07-04", end_time="9999-12-31",rename=None):
  """return daily time point
  start time: 1776-07-04 is the earliest time
  end time: 9999-12-31 (latest available)
  """
  store_list = []
  URL = f"https://api.stlouisfed.org/fred/series/observations?"
  params = {
      "api_key":api_key,
      "series_id":serie_id,
      "file_type": "json",
      "observation_start": start_time,
      "observation_end": end_time,

  }
  response = requests.get(URL, params=params )
  if response.status_code == 200:
    data = response.json()
    observations = data.get("observations")
    for ob in observations:
      store_list.append({"Date":ob.get("date"), "Value":ob.get("value")})
  else:
      print(f"Error: {response.status_code}")
      return

  res_df = pd.DataFrame(store_list)
  #  Replace '.' with NaN
  res_df['Value'] = pd.to_numeric(res_df['Value'], errors='coerce')
  # Drop rows with NaN in 'value'
  res_df.dropna(subset=['Date','Value'], inplace=True)
  # set DatatimeIndex
  res_df['Date'] = pd.to_datetime(res_df['Date'])
  res_df['Value'] = res_df["Value"].astype(float)
  res_df = res_df.set_index('Date')
  res_df = res_df.sort_index()
  if rename:
    res_df.rename(columns={"Value":rename}, inplace=True)
  return res_df


def get_serie_info(series_name, api_key):
  store_list = []
  URL = f"https://api.stlouisfed.org/fred/series?"
  params = {
      "api_key":api_key,
      "series_id":series_name,
      "file_type": "json",
  }
  response = requests.get(URL, params=params )
  if response.status_code == 200:
    data = response.json()
    data = data.get("seriess")[0]
    df=pd.DataFrame.from_dict(data, orient='index', columns=['Value'])
  else:
      print(f"Error: {response.status_code}")
      return
  print(df)
  return df


## B. Find series name

In [ ]:
df=search_series("unemployment+rate", fred_api_key, "Monthly", "Unemployment Rate")

In [ ]:
df.head()

In [ ]:
df=search_series("M2", fred_api_key, "Monthly")

In [ ]:
df.head()

# df.title.values

In [ ]:
df=get_serie_info("CIVPART", fred_api_key)
df.loc['notes'].values

Here is the list of serie name I collected:
CPIAUCSL: Consumer Price Index for All Urban Consumers (target) \

//GDP
GDP: gross domestic product

//labor market
UNRATE: unemployment \
CIVPART: Labor Force Participation Rate \
PAYEMS: Total Nonfarm Payrolls

//consumption
PCEPI: Personal Consumption Expenditures Price Index\
PCEPILFE: Personal Consumption Expenditures Excluding Food and Energy (Chain-Type Price Index) \
RSAFS: "Advance Retail Sales: Retail Trade and Food Services

//production
PPIACO: Producer Price Index for All Commodities

//Money supply
FEDFUNDS: Federal Funds Effective Rate \
M2SL: M2 Money Stock \
WTISPLC: Spot Crude Oil Price: West Texas Intermediate (WTI) \
UMCSENT: University of Michigan: Consumer Sentiment \
HOUST: New Privately-Owned Housing Units Started: Total Units\
NM09075USM476NNBR: onfarm Real Estate Foreclosures for United States


## C. Data collection: get time series data for each serie name

In [ ]:
series_name_dict={
            "UNRATE":" unemployment",
            "CIVPART": "Labor Force Participation Rate",
            "PAYEMS":"Total Nonfarm Payrolls",
            "FEDFUNDS": "Federal Funds Effective Rate (daily rate)",
            "M2SL": "M2 Money Stock",
            "HOUST": "New Privately-Owned Housing Units Started: Total Units",
            "UMCSENT":" University of Michigan: Consumer Sentiment",

            "WTISPLC": "Spot Crude Oil Price: West Texas Intermediate (WTI)",
            "PPIACO": "Producer Price Index for All Commodities",
            "PCEPI": "Personal Consumption Expenditures Price Index",
            "PCEPILFE": "Personal Consumption Expenditures Excluding Food and Energy (Chain-Type Price Index)",
            "CPIAUCSL": "Consumer Price Index for All Urban Consumers",
            "CPILFESL": "(Core CPI)Consumer Price Index for All Urban Consumers: All Items Less Food and Energy in U.S. City Average"

}
#  "RSAFS": "Advance Retail Sales: Retail Trade and Food Services",
# NM09075USM476NNBR: onfarm Real Estate Foreclosures for United States,

In [ ]:
df = pd.DataFrame()
for serie_name, serie_description in series_name_dict.items():
  cur_df=get_time_series(serie_name, fred_api_key, rename=serie_name)
  # print(serie_name, cur_df.shape)
  df[serie_name]=cur_df
print(df.shape)

remove row with NaN value

In [ ]:
df.dropna(inplace=True)
print(df.shape)

Calculate rate of change over  yearly period

In [ ]:
period=12
df["fedfund_rate"]=(df['FEDFUNDS'].pct_change(periods=period)*100)*round(2)
df['unemployment_rate'] =  (df['UNRATE'].pct_change(periods=period)*100)*round(2)
df['CIVPART_rate'] =  (df['CIVPART'].pct_change(periods=period)*100)*round(2)
df['inflation_rate']=(df.CPIAUCSL.pct_change(periods=period)*100)*round(2)
df['core_inflation_rate']=(df.CPILFESL.pct_change(periods=period)*100)*round(2)
df['housing_rate'] = (df.HOUST.pct_change(periods=period)*100)*round(2)
df['PPIACO_rate'] = (df['PPIACO'].pct_change(periods=period)*100)*round(2) #commodity index
df['MoneySupply_rate'] = (df['M2SL'].pct_change(periods=period)*100)*round(2)
df['oilprice_rate'] = (df['WTISPLC'].pct_change(periods=period)*100)*round(2)
# df['UMCSENT_rate'] =  (df['UMCSENT'].pct_change(periods=period)*100)*round(2)
df.shape

In [ ]:
# df[df.index.year==2022]
df.tail()

In [ ]:
# df.info()

Note: CPI is an index,  relative to the average price level during the 1982–1984 period, which is set to 100. There is value before 1982 because it's calculated backwards. \
Save the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
df.to_csv("/content/drive/My Drive/economic_data_observation.csv")

# II. Predicting Inflation

## A. Using Linear Regression Model

### A1. Use heatmap to evaluate relaionship of inflation with other variables

In [ ]:
df_=df[~df.inflation_rate.isna()][['HOUST', "housing_rate",'UNRATE', 'CIVPART', "CIVPART_rate", 'FEDFUNDS', 'UMCSENT', "PPIACO_rate","MoneySupply_rate", "oilprice_rate","inflation_rate", "core_inflation_rate"]]
# df_=df[~df.inflation_rate.isna()]

correlation_matrix = df_.corr()
# create heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".1f", vmin=-1, vmax=1)
plt.title('Matrix Heatmap')
plt.show()

Inflation rate is positively correlated with oil_price rate , moneysupply rate, PPIACO rate, FEDFUNDS, and civpart_rate \
Now I use Cross Correlation to identify which lag period for each variable show strong correlation



### A2. Cross Correlation and identify shift lag period
Cross Correlation is a method used to measure the similarity between two time series signals

In [ ]:
df[['FEDFUNDS', 'inflation_rate']].plot(figsize=(12, 6))
plt.title("Federal Rate vs Inflation rate over time")
plt.show()

Does inflation happen after or before a change in FedFund rate? \
We know that change in FedFund on inflation is not immediate, there is typically a lag. \
We use cross correlation to identify which lag time show the highest correlation

In [ ]:
df_=df[~df.inflation_rate.isna()]
df_.shape

In [ ]:
lags = range(-15, 15)  # lag of ±15 months
cross_cors = [df_['FEDFUNDS'].corr(df_['inflation_rate'].shift(lag)) for lag in lags] #shifting inflation by various lag to compare to FedFunds

fig, ax = plt.subplots(figsize=(10, 5))

ax.bar(lags, cross_cors, color='purple')
ax.set_title('Cross-correlation between Fed Funds Rate and Inflation Rate')
ax.set_xlabel('Lag (months)')
ax.set_ylabel('Correlation coefficient')
# ax.xaxis.set_major_locator(MultipleLocator(1))
ax.set_xticks(lags)
ax.tick_params(axis='x', labelrotation=45)
plt.show()

In [ ]:
lags = range(-15, 15)  # lag of ±15 months
cross_cors = [df_['MoneySupply_rate'].corr(df_['inflation_rate'].shift(lag)) for lag in lags]
# cross_cors = [df_['M2SL'].corr(df_['inflation_rate'].shift(lag)) for lag in lags]

fig, ax = plt.subplots(figsize=(10, 5))

ax.bar(lags, cross_cors)
ax.set_title('Cross-correlation between Money Supply Rate and Inflation Rate')
ax.set_xlabel('Lag (months)')
ax.set_ylabel('Correlation coefficient')
ax.set_xticks(lags)
ax.tick_params(axis='x', labelrotation=45)
plt.show()

In [ ]:
lags = range(-15, 15)  # lag of ±15 months
cross_cors = [df_['PPIACO_rate'].corr(df_['inflation_rate'].shift(lag)) for lag in lags]
fig, ax = plt.subplots(figsize=(10, 5))

ax.bar(lags, cross_cors, color='red')
ax.set_title('Cross-correlation between CPI Rate and Inflation Rate')
ax.set_xlabel('Lag (months)')
ax.set_ylabel('Correlation coefficient')
ax.set_xticks(lags)
ax.tick_params(axis='x', labelrotation=45)

plt.show()

In [ ]:
lags = range(-15, 15)  # lag of ±15 months
cross_cors = [df_['CIVPART_rate'].corr(df_['inflation_rate'].shift(lag)) for lag in lags]
fig, ax = plt.subplots(figsize=(10, 5))

ax.bar(lags, cross_cors, color="green")
ax.set_title('Cross-correlation between Civil Participation Rate and Inflation Rate')
ax.set_xlabel('Lag (months)')
ax.set_ylabel('Correlation coefficient')
ax.set_xticks(lags)
ax.tick_params(axis='x', labelrotation=45)

plt.show()

In [ ]:
lags = range(-15, 15)  # lag of ±24 months
cross_cors = [df_['oilprice_rate'].corr(df_['inflation_rate'].shift(lag)) for lag in lags]
fig, ax = plt.subplots(figsize=(10, 5))

ax.bar(lags, cross_cors,color="black")
ax.set_title('Cross-correlation between Oil Rate and Inflation Rate')
ax.set_xlabel('Lag (months)')
ax.set_ylabel('Correlation coefficient')
# ax.xaxis.set_major_locator(MultipleLocator(1))
ax.set_xticks(lags)
ax.tick_params(axis='x', labelrotation=45)

plt.show()

In [ ]:
From the correlation plot, we see that Oil Rate: correlation peaks at lag=-1 => the change in oil rate of previous month is reactive with the inflation \
PPICAO Rate : correlation peaks at lag=-1 => the change in commodity index of previous month is reactive with the inflation \
Money Supply Rate: peaks at lag -12: change in money rate of previous year is reactive with inflation \
Civil Rate participate : lag =0 \
fedFund: peak at lag =12 => this makes sense becasue Policymakers need to observe sustained inflation, assess its causes, and go through policy-setting procedures, Wants to avoid overreacting to short-term fluctuations, so they often respond gradually.

### A3. Use Linear Regression Model to predict inflation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df_shift = df.copy()
selected_columns =[ "CIVPART_rate", 'FEDFUNDS', 'UMCSENT', "PPIACO_rate","MoneySupply_rate", "oilprice_rate","inflation_rate"]
df_shift = df_shift[selected_columns]
df_shift["CIVPART_rate_lag"]=df_shift["CIVPART_rate"].shift(0)
df_shift["FEDFUNDS_lag"]=df_shift["FEDFUNDS"].shift(12)
df_shift["PPIACO_rate_lag"]=df_shift["PPIACO_rate"].shift(-1)
df_shift["MoneySupply_rate_lag"]=df_shift["MoneySupply_rate"].shift(-12)
df_shift["oilprice_rate_lag"]=df_shift["oilprice_rate"].shift(-1)
df_shift.dropna(axis=0,inplace=True)
df_shift.shape

In [ ]:
selected_columns =[ "CIVPART_rate_lag", 'FEDFUNDS_lag', "PPIACO_rate_lag", "MoneySupply_rate_lag", "oilprice_rate_lag","inflation_rate"]
df_=df_shift[selected_columns].dropna()
df_.shape

In [ ]:
target_col="inflation_rate"
X=df_.drop(target_col,axis=1)
y=df_[target_col]
year = "2017"
X_train = X.loc[X.index<year]
y_train = y.loc[y.index<year]

X_test = X.loc[X.index >=year]
y_test = y.loc[y.index >= year]

model = LinearRegression()
model.fit(X_train, y_train)

y_pred_ = model.predict(X_test)
y_pred = pd.Series(y_pred_, index=y_test.index)

print("MSE:", mean_squared_error(y_test, y_pred))
print("R^2 Score:", model.score(X_test, y_test))

The linear regression model does not perform well with inflation prediction.
=> understand why \


### A4. Evaluate linear assumption

#### Relationship between the independent and dependent variables is linear.

In [ ]:
import seaborn as sns
target_col="inflation_rate"
features_df=df_.drop(target_col,axis=1)
features= features_df.columns
sns.pairplot(df_, x_vars=features, y_vars=target_col, kind='reg' )

#### Normality of Residuals

Residuals (the differences between observed and predicted values) should follow a normal distribution when considering multiple predictors together.

In [ ]:
import numpy as np
from scipy.stats import norm

residuals = y_test - y_pred
# Plot histogram of residuals
count, bins, ignored = plt.hist(residuals, bins=30, density=True, alpha=0.6)

#normal distribution line over the distribution of residuals

mean = np.mean(residuals)
std = np.std(residuals)

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mean, std)
plt.plot(x, p, 'r', linewidth=2)

plt.title('Histogram of Residuals with Normal Distribution Fit')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()

#### homoscedasticity (constant variance) of the errors versus predictions



In [ ]:
plt.scatter(y_pred, residuals)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.title("Residuals vs Predicted")
plt.show()

The residuals are not evenly scattered.

#### statistical independence of the errors:  Use Durbin-Watson test

In [ ]:
from statsmodels.stats.stattools import durbin_watson

dw = durbin_watson(residuals)
print(f'Durbin-Watson: {dw}')

## B. ARIMA model

natural model for autocorrelated data is the ARIMA model \
ARIMA models assume the time series data is stationary. \
Stationary means statistical properties of time series (mean and variance) do not change over time.

In [ ]:
plt.plot(df.index, df.inflation_rate)
plt.title("Inflation rate over time")
plt.xlabel("Years")
plt.ylabel("Inflation Rate (%)")
plt.gca().xaxis.set_major_locator(mdates.YearLocator(5))  # every year
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

The time series plot show trends over time, suggesting nonstationary. To verify, I use adfuller test and ACF plot (autocorrelation Function plot). The ACF plot show correlation of time series with lagged version, showing how current value is related to past value. \
Augmented Dicky-Fuller (ADF) test is statistical tst to deterine if a time series is stationary.

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(df['inflation_rate'].dropna())  # dropna in case there are NaNs

print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:', result[4])

ADF statistic (-2.51) is higher (less negative) than all critical values (especially the 5% and 1% levels), and the p-value (0.1135) > 0.05.
The data is non stationary

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose, STL
result = seasonal_decompose(df['inflation_rate'].dropna(), model='additive', period=12)
result.plot()

# stl = STL(df['inflation_rate'].dropna(), period=12)
# result = stl.fit()

# ig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 8), sharex=True)
# ax1.plot(df['inflation_rate'])
# ax1.set_title('Original Sales Data')
# ax2.plot(result.trend)
# ax2.set_title('Trend Component')
# ax3.plot(result.seasonal)
# ax3.set_title('Seasonal Component')
# ax4.plot(result.resid)
# ax4.set_title('Residual (Noise) Component')

# plt.tight_layout()
# plt.show()

Data is seasonal => use SARIMA

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plot_acf(df['inflation_rate'].dropna(), lags=4*12)
plot_pacf(df['inflation_rate'].dropna(),lags=4*12)
# plot_acf(df.resample('Y').median().dropna().inflation_rate, lags=20)

In [ ]:
 The ACF and PACF plots show significant autocorrelation at the first lags, and not decay to 0, suggesting data is not stationary and requires differencing.

 The lag where PACF cuts off → candidate P = 1

The lag where ACF cuts off → candidate Q=36 => 3


find optimal d for Arima model

In [ ]:
d =1
inflation_rate = df['inflation_rate'].dropna()
p_value = 1
while p_value > 0.05:
  new_inflation_rate = df['inflation_rate'].diff().dropna()
  result = adfuller(new_inflation_rate)
  p_value = result[1]
  print(d)
  print('ADF Statistic:', result[0])
  print('p-value:', result[1])

In [ ]:
new_inflation_rate = df['inflation_rate'].diff(12).dropna()
result = adfuller(new_inflation_rate)
p_value = result[1]

print('p-value:', result[1])

In [ ]:
the optimal d is 1

In [ ]:
import statsmodels.api as sm

p, d, q = 3, 1, 1           # non-seasonal orders
P, D, Q, s = 1, 1, 3, 12   # seasonal orders

model = sm.tsa.statespace.SARIMAX(df['inflation_rate'].dropna(),
                                  order=(p, d, q),
                                  seasonal_order=(P, D, Q, s),
                                  enforce_stationarity=False,
                                  enforce_invertibility=False)

results = model.fit()

print(results.summary())
